In [8]:
import re

https://www.regular-expressions.info/catastrophic.html

# Processando textos reais

Vamos trabalhar com um conjunto de textos obtidos da Wikipedia em portugues (https://dumps.wikimedia.org/ptwiki/20210301/). O conjunto original de dados é bem grande (1.8 GB compactado), então eu criei um arquivo reduzido da seguinte forma:

- Li os registros em formato XML (é assim que estão armazenados no *dump*)

- Removi todos os registros que não diziam respeito a um artigo, mas que eram referentes a redirecionamento de pagina e outras meta-informações.

- Removi todos os campos de cada registro exceto titulo e corpo.

- Selecionei apenas $1\%$ dos registros restantes.

- Gravei cada registro como uma linha de texto no formato JSON em um arquivo dump_small.jsonln ("JSON lines")

Vamos ler esse arquivo e ver seu conteudo:

In [3]:
import json

data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Numero de documentos: {len(data)}')

Numero de documentos: 11225


In [4]:
print(data[0].keys())

dict_keys(['title', 'body'])


In [5]:
print(data[0]['title'])

Alexandre (nome)


In [6]:
#print(data[0]['body'])

---

Como vocês podem ver, o texto está cheio de caracteres de estruturação de documento.

---

**Atividade:** Consulte a documentação da Wikipedia para descobrir o que são as marcações `[[`, `]]`, `{{` e ```}}```

**R:** 
https://en.wikipedia.org/wiki/Help:Wikitext


- [[ ]] => link para outra página da Wikipédia (fazer um hiperlink)
- {{ }} => são templates que substitui por rum html desejado

---

Vamos explorar o corpus para minerar itens de interesse.

**Exercicio:** Faça uma função que retorna uma lista com todos os links html de um texto deste corpus.

In [12]:
# Regex obtida de https://www.geeksforgeeks.org/python-check-url-string/
pattern = r"""
    (?i)  # Ignore case.
    \b  # Inicio de palavra.
    (?:
        https?://
    |
        www
        \d{0,3}
        [.]
    |
        [a-z0-9.\-]+
        [.]
        [a-z]{2,4}
        /
    )
    (?:
        [^\s()<>]+
    |
        \(
        (?:
            [^\s()<>]+
        |
            \(
            [^\s()<>]+
            \)
        )*
        \)
    )+
    (?:
        \(
        (?:
            [^\s()<>]+
        |
            \(
            [^\s()<>]+
            \)
        )*
        \)
    |
        [^\s`!()\[\]{};:'\".,<>?«»“”‘’]
    )
"""
matcher = re.compile(pattern, re.VERBOSE)

In [15]:
for item in data[:10]:
    texto = item['body']
    for match in matcher.findall(texto):
        print(match)

http://www.geocities.com/kurogr/linearb.pdf
https://web.archive.org/web/20080627021042/http://www.geocities.com/kurogr/linearb.pdf
http://www.cm-aveiro.pt
www.cm-aveiro.pt
http://ww3.aeje.pt/avcultur/avcultur/ArkivDtA/Vol02/Vol02p081.htm|autor=Francisco
https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_unid_territorial&menuBOUI=13707095&contexto=ut&selTab=tab3|titulo=Statistics
www.ine.pt|lingua=en
https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_unid_territorial&menuBOUI=13707095&contexto=ut&selTab=tab3|titulo=Statistics
www.ine.pt|lingua=en
http://www.ine.pt/ngt_server/attachfileu.jsp?look_parentBoui=379490&att_display=n&att_download=y
http://www.ine.pt/investigadores/Quadros/Q101.zip
http://www.dgterritorio.pt/ficheiros/cadastro/caop/caop_download/caop_2013_0/areasfregmundistcaop2013_2
https://dre.pt/application/dir/pdf1s/2013/01/01901/0000200147.pdf|titulo=Lei
http://www.ordens.presidencia.pt/?idc=153
https://www.academia.edu/15680102/The_fable_of_the_cod_and_the_promised_sea.

In [14]:
def links_html(data):
    list_link = []
    href = re.compile(r"(?<=\[\[).*(?=\]\])")  
    matches = href.finditer(data)
    for match in matches:
        list_link.append(match.group())
        
    return list_link
        
# links_html(data[0]["body"])

**Exercicio:** Faça uma função que recebe um texto do corpus e substitui todas as ocorrências de links da Wikipedia (textos entre os tags `[[` e `]]`) por texto simples, e retorna o documento limpo. Por exemplo, quando encontrar algo como `[[Etimologia|Etimologicamente]]` substituir por `Etimologicamente`. Você pode supor que os links não são *aninháveis*, como por exemplo `[[blablabla[[etc]]blebleble]]`.

In [16]:
texto = 'blalalal [[nejfeifd|fdf]] gf [[var]] vfdvd'
texto = data[0]["body"]

In [17]:
#lazy Vs greedy capture

#greedy capture: pgear o maximo
pattern = r"\[\[(.*)\]\]" # ? até a primeira captura
# re.findall(pattern, texto)

#lazy capture
#localizar o que está entre barras
pattern = r"\[\[(.*?)\]\]" # ? até a primeira captura
# re.findall(pattern, texto)

In [19]:
#queremos subtituir
# 14.32 pm
def limpa_wikilinks(text):
    pattern = r"\[\[(?:[^|]*?\|)*?([^|]*?)\]\]"
    repl = r'\1'
    matcher = re.compile(pattern)
    return matcher.sub(repl, texto)

In [20]:
def limpa_aspas(texto):
    pattern = r"""(['"]+)(.*?)\1"""
    repl = r"\2"
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

**Exercicio**: Faça uma função que recebe um texto do corpus e remove todas as ocorrências de referências (textos entre os tags `<ref>` e `</ref>`).

In [ ]:
dado = data[0]["body"]

In [143]:
def clean_ref(dado):
    ref = re.compile(r"(?<=.)<ref>.*</ref>(?=)")
    matches = ref.finditer(dado)
    for match in matches:
        dado= dado.replace(match.group(), "")
    return dado

# clean_ref(data[0]["body"])

In [21]:
def limpa_ref(texto):
    pattern = r"""<ref>.*?</ref>"""
    repl = r""
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

In [22]:
def limpa_url(texto):
    # Regex obtida de https://www.geeksforgeeks.org/python-check-url-string/
    pattern = r"""
        (?i)  # Ignore case.
        \b  # Inicio de palavra.
        (?:
            https?://
        |
            www
            \d{0,3}
            [.]
        |
            [a-z0-9.\-]+
            [.]
            [a-z]{2,4}
            /
        )
        (?:
            [^\s()<>]+
        |
            \(
            (?:
                [^\s()<>]+
            |
                \(
                [^\s()<>]+
                \)
            )*
            \)
        )+
        (?:
            \(
            (?:
                [^\s()<>]+
            |
                \(
                [^\s()<>]+
                \)
            )*
            \)
        |
            [^\s`!()\[\]{};:'\".,<>?«»“”‘’]
        )
    """
    repl = ''
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

**Exercicio:** Faça uma função que recebe um texto do corpus e remove todos os templates (textos entre os tags `{{` e `}}`). Este exercício é desafiante: *os templates podem ser aninhados*! E agora, como proceder?

In [23]:
def limpa_templates(texto):
    conta = 0
    spans_proibidos = []
    for item in re.finditer(r'{{|}}', texto):
        if item[0] == '{{':
            if conta == 0:
                inicio = item.span()[0]
            conta += 1
        else:
            conta -= 1
            if conta == 0:
                fim = item.span()[1]
                spans_proibidos.append((inicio, fim))
    texto_limpo = ''
    inicio = 0
    for span in spans_proibidos:
        fim, novo_inicio = span
        texto_limpo += texto[inicio:fim]
        inicio = novo_inicio
    texto_limpo += texto[inicio:]
    return texto_limpo

def limpa_texto(texto):
    return limpa_url(limpa_templates(limpa_aspas(limpa_wikilinks(limpa_ref(texto)))))

In [24]:
for item in data[:10]:
    texto = item['body']
    print(limpa_texto(texto))
    print('=' * 80)



Alexandre é um prenome popular da língua portuguesa. É cognato ao nome Alexander, da língua inglesa. Em países lusófonos, pessoas chamadas Alexandre são normalmente apelidadas de Alex.

== Origem ==
O nome deriva do latim Alexander, romanização do nome grego Αλέξανδρος (Aléksandros). Etimologicamente, o nome seria um composto do verbo ἀλέξειν (aléksein), defender, e o substantivo ἀνδρός (andrós), genitivo de ἀνήρ (anēr), homem. Assim, pode ser traduzido como protetor da Humanidade.

O termo seria ou um tipo raro de composto tatpurusha invertido, com o modificante na segunda posição (o tatpurusha cognato em sânscrito sendo *nararakṣa, cf. Ramayana 6.33.45; o equivalente exato em sânscrito seria *rakṣinara, do PIE hleks(i)-hnros) ou um composto gasto do tipo terpsimbrotos, cujo significado original seria ele protege os homens.

O primeiro registro conhecido do nome foi feito no grego micênico: encontrou-se a versão feminina do nome, Alexandra, escrito em Linear B.<ref></ref><ref>Chadwi

**Exercicio:** Usando os codigos desenvolvidos acima, faça uma função que recebe um texto do corpus e retorna a sua versão limpa.

**Exercicio:** Limpe todos os documentos e explore os resultados para ver o que mais dá para limpar. Nosso objetivo é ter uma coletânea de textos limpos para poder criar um vocabulário da língua portuguesa!

# Criando um tokenizador

Agora que temos um conjunto de documentos limpos, vamos analisar esses documentos para:

- Separar os documentos em palavras

- Separar os documentos em frases

**Exercicio:** Faça uma função que separa os documentos em palavras usando simplesmente os espaços em branco como indicador de separação.

In [146]:

documents = data[0]['body']

def separaEspaco(texto):
    palavras = texto.split(' ')
    return palavras
# separaEspaco(documents)

**Exercicio:** Agora analise os resultados obtidos: essa separação em palavras está perfeita? Certamente não:

- Um problema óbvio é o seguinte: palavras ao final de uma sentença podem carregar consigo a pontuação da frase. 

- Mais ainda, algumas palavras no meio das sentenças podem ter vírgula e ponto-e-vírgula juntos.

- Sequências de emojis podem aparecer, e cada emoji deve ser considerado como uma palavra separada

Faça uma função que recebe um documento e faz uma separação melhor.

In [148]:
def separacaoMelhor(comPontuacao):
    semPontuacao = []
    for palavra in comPontuacao:
        palavrasSemPontuacao = re.sub(r"[^\w\s]|\n", "", palavra)
        semPontuacao.append(palavrasSemPontuacao)
    return semPontuacao

# separacaoMelhor(separaEspaco(documents))
